<a href="https://colab.research.google.com/github/jpdicarvalho/POO-Python/blob/main/Projeto_Elei%C3%A7%C3%B5es_de_Condom%C3%ADnio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Eleição de Condomínio**
# **Author: João Pedro Braga de Carvalho**
---
## **Documentação do Projeto: Organizando uma Eleição**
Este documento descreve a estrutura e a funcionalidade do sistema eletrônico desenvolvido para auxiliar na votação do novo síndico em um condomínio.

#Estrutura de Classes

## Morador
Representa cada pessoa que habita no condomínio.
1. **Atributos**: nome, apartamento.
2. **Métodos**:
- **Construtor**: recebe opcionalmente um nome e um objeto Apartamento. Se não receber um nome, solicita através de input. Se não receber um objeto Apartamento, solicita o número do apartamento e cria o objeto.
- **Votar**: recebe um objeto Urna e um número de candidato (opcional) e registra o voto na Urna.

In [5]:
class Morador():
    def __init__(self, nome, num_apartamento):
        self.nome = nome
        self.num_apartamento = num_apartamento

    def votar(self, urna, num_apartamento):
        num_candidato_votado = input("\nDigite o número do candidato em que deseja votar (ou digite 0 para sair): ")
        if num_candidato_votado == '0':
            return  # Encerra a função se o usuário digitar 0

        num_candidato_votado = int(num_candidato_votado)  # Convertendo para inteiro

        for apartamento in urna.listaApartamentos.values():
            if 'Candidato' in apartamento:
                if apartamento['numCandidatura'] == num_candidato_votado:
                    apartamento['votosRecebidos'] += 1

        quemVotou = urna.listaApartamentos[self.num_apartamento]
        if not quemVotou['voto']:
            quemVotou['voto'] = True
            print("Voto registrado com sucesso!")
            return


        print("Número de candidatura não encontrado. Por favor, tente novamente.")

## Candidato
É um tipo especial de Morador.
1. **Atributos**: número (identificação na Urna), contagem de votos.
2. **Método**:
- **Construtor**: inicializa com nome e apartamento do Morador e atribui 0 à contagem de votos.


In [6]:
class Candidato(Morador):
    def __init__(self, nome, num_apartamento, numCandidatura):
        super().__init__(nome, num_apartamento)
        self.numCandidatura = numCandidatura
        self.votosRecebidos = 0

## Apartamento
Representa as unidades do prédio.
1. **Atributos**: lista de moradores, status de voto.
2. **Métodos**:
- **Adicionar Morador**: adiciona um objeto Morador à lista.
- **Visualizar Moradores**: exibe a lista de moradores.
- **Atualizar Status de Voto**: marca o apartamento como votado.

In [8]:
class Apartamento():
    def __init__(self, numApartamento):
        self.numApartamento = numApartamento

## Urna
Responsável pela lógica da eleição.
1. **Atributos**: coleção de apartamentos, coleção de candidatos.
2. **Métodos**:
- **Cadastro de Apartamentos e Candidatos**: adiciona objetos existentes à coleção e gera número único para candidatos.
- **Receber Voto**: atualiza a contagem de votos do candidato e marca o apartamento como votado.
- **Sinalizar Encerramento**: indica se a votação terminou.

In [7]:
class Urna():
    def __init__(self, listaApartamentos={}):
        self.listaApartamentos = listaApartamentos

    def gerarNumCandidato(self):
        num_candidato = 1001  # Número inicial para os candidatos
        for apartamento in self.listaApartamentos.values():
            if 'Candidato' in apartamento:  # Verifica se há um candidato neste apartamento
                apartamento['numCandidatura'] = num_candidato
                num_candidato += 1

    def listarApartamentos(self, listaApartamentos={}):
        print("\n#Confira abaixo, os apartamentos cadastrados para votar:")
        for num in self.listaApartamentos.keys():
          print(f">> Apartamento Nº {num}")

    def listarCandidatos(self, listaApartamentos={}):
        print("\n#Confira abaixo, os candidatos disponíveis:")
        for num_apartamento, detalhes in self.listaApartamentos.items():
            if 'Candidato' in detalhes:
                nome_candidato = detalhes['Candidato']
                num_candidatura = detalhes['numCandidatura']
                print(f">> Candidato: {nome_candidato}, Número de Candidatura: {num_candidatura}")

# Programa Principal
O programa principal possui 3 etapas:

1. **Cadastro**: Lê informações sobre moradores e cria os objetos Morador/Candidato e Apartamento correspondentes.
2. **Configuração**: Utiliza as informações obtidas para criar uma Urna.
3. **Votação**: Permite que os moradores votem. Encerra quando a Urna informa que a votação acabou.

In [ ]:
def main():
    listaApartamentos = {}

    while True:
        tipo = input("Digite M para cadastrar um Morador e C para cadastrar um Candidato (ou digite qualquer outra tecla para sair): ")

        if tipo.upper() == 'M':
            nome = input('Nome do morador: ')
            num = input('Qual o número do apartamento? ')
            morador = Morador(nome, num)
            listaApartamentos[num] = {
                'Morador': morador.nome,
                'voto': False
            }
            print("\nApartamento cadastrado com sucesso!")

        elif tipo.upper() == 'C':
            nome = input('Nome do Candidato: ')
            num = input('Qual o número do apartamento? ')
            candidato = Candidato(nome, num, 0)
            listaApartamentos[num] = {
                'Candidato': candidato.nome,
                'numCandidatura': 0,
                'votosRecebidos': 0,
                'voto': False
            }
            print("\nApartamento cadastrado com sucesso!")

        else:
            print("Opção inválida. Encerrando o cadastro.")
            break

    urna = Urna(listaApartamentos)
    urna.gerarNumCandidato()

    print("\n====== Cadastro concluído. Vamos começar a eleição! =====")

    while True:
      allVote = True

      urna.listarApartamentos(listaApartamentos)
      print(listaApartamentos)

      votoApartamento = input(f'\nInforme o número do apartamento que irá votar: ')
      if votoApartamento not in listaApartamentos:
          print("XXXXX Número de apartamento inválido. Por favor, tente novamente. XXXXX")
          continue

      detalhes = listaApartamentos[votoApartamento]
      if detalhes['voto']:
          print("XXXX Este apartamento já votou. XXXX")
          continue

      print("+++ Apartamento habilitado a votar! +++")
      urna.listarCandidatos(listaApartamentos)

      morador = Morador(detalhes.get('Morador', detalhes.get('Candidato')), votoApartamento)
      morador.votar(urna, votoApartamento)

      for apartment, valueVote in listaApartamentos.items():
          if not valueVote['voto']:  # Verifica se algum voto é False
              allVote = False
              break

      if allVote:  # Se todos os votos forem True, encerra a votação
          break


    print("\nApuração dos votos:")
    candidato_vencedor = None
    votos_vencedor = 0
    for num_apartamento, detalhes in listaApartamentos.items():
        for chave, valor in detalhes.items():
            if 'Candidato' in detalhes:
                votos_recebidos = detalhes['votosRecebidos']
                if votos_recebidos > votos_vencedor:
                    candidato_vencedor = valor
                    votos_vencedor = votos_recebidos

    if candidato_vencedor:
        print(f"Candidato vencedor: {candidato_vencedor}, Votos Recebidos: {votos_vencedor}")

if __name__ == "__main__":
    main()